# 1. Preprocessing
## a. Dataset Description
Provide a brief description of the dataset. What are the attributes, what is the target attribute? What is the purpose of the dataset? Specify which attributes are discrete and continuous.
## b. Handling NaN values
Identify the NaN's (Not a Number) in your dataset. Remove the rows that contain such values.
## c. Statistics
Calculate the mean and variance for each numerical attribute.
## d. Remove the target attribute
Remove the target attribute from your dataset.

In [145]:
import pandas as pd
import numpy as np



# Dataset
dataFrame = pd.read_csv('https://raw.githubusercontent.com/Vali2804/ML_setDate/main/penguins.csv')

# 1. Preprocessing
# Dataset Description 

#a.
# Penguin Dataset Description
attributes = ['species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
targetAttribute = 'species'
purpose = 'Classification'

# Attribute types
discreteAttributes = ['species', 'island','sex']
continuousAttributes = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

# b.
# Remove NaN values
dataFrame = dataFrame.dropna()

# c.
# Calculte mean and variance for each attribute
num_attributes = dataFrame[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]
mean_values = num_attributes.mean()
variance_values = num_attributes.var()

print('Mean values for each attribute:\n', mean_values)
print('Variance values for each attribute:\n', variance_values)

# d.
# Remove the target attribute from the dataset
dataFrame = dataFrame.drop(columns=['species'])
print('Dataset without target attribute:\n', dataFrame)


Mean values for each attribute:
 bill_length_mm         43.992793
bill_depth_mm          17.164865
flipper_length_mm     200.966967
body_mass_g          4207.057057
dtype: float64
Variance values for each attribute:
 bill_length_mm           29.906333
bill_depth_mm             3.877888
flipper_length_mm       196.441677
body_mass_g          648372.487699
dtype: float64
Dataset without target attribute:
         island  bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  \
0    Torgersen            39.1           18.7              181.0       3750.0   
1    Torgersen            39.5           17.4              186.0       3800.0   
2    Torgersen            40.3           18.0              195.0       3250.0   
4    Torgersen            36.7           19.3              193.0       3450.0   
5    Torgersen            39.3           20.6              190.0       3650.0   
..         ...             ...            ...                ...          ...   
338     Biscoe            

# 2. Distances
## a. Data Preprocessing - Handling Non-Numeric Discrete Attributes
- Convert the discrete attributes that are not numeric (such as strings or boolean values) into numerical. If this doesn't apply to your dataset, provide
a short explanation on how you would proceed.


In [146]:
# a. convert discrete attributes to numeric
dataFrame['island'] = dataFrame['island'].astype('category').cat.codes
dataFrame['sex'] = dataFrame['sex'].astype('category').cat.codes

print('Dataset with numeric discrete attributes:\n', dataFrame)

Dataset with numeric discrete attributes:
      island  bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  \
0         2            39.1           18.7              181.0       3750.0   
1         2            39.5           17.4              186.0       3800.0   
2         2            40.3           18.0              195.0       3250.0   
4         2            36.7           19.3              193.0       3450.0   
5         2            39.3           20.6              190.0       3650.0   
..      ...             ...            ...                ...          ...   
338       0            47.2           13.7              214.0       4925.0   
340       0            46.8           14.3              215.0       4850.0   
341       0            50.4           15.7              222.0       5750.0   
342       0            45.2           14.8              212.0       5200.0   
343       0            49.9           16.1              213.0       5400.0   

     sex  
0      1 

## b. Minkowski Distance 
Write a function distance_points that calculates the distance between two points. The function should take three parameters: the two points and
p , where p indicates the order of the Minkowski distance (remember that p=1 is the equivalent for the Manhattan distance, and p=2 for the
Euclidean one).

In [147]:
def distance_points(point1, point2, p):
    if len(point1) != len(point2):
        raise ValueError("Points must have the same dimensionality")

    distance = 0
    for i in range(len(point1)):
        distance += abs(point1[i] - point2[i]) ** p

    distance = distance ** (1/p)
    return distance

point_a = [1, 2, 3]
point_b = [4, 5, 6]
p_value = 2

# testing the function
distance = distance_points(point_a, point_b, p_value)
print(f"Minkowski Distance between {point_a} and {point_b} (p={p_value}): {distance}")


Minkowski Distance between [1, 2, 3] and [4, 5, 6] (p=2): 5.196152422706632


## c. The function calculate_distance_matrix
- Write a function calculate_distance_matrix that creates a lower triangular matrix M , where M[i,j] contains the distance between the instances
i and j from the dataset.


In [148]:
def calculate_distance_matrix(dataset, p):
    num_instances = len(dataset)
    distance_matrix = np.zeros((num_instances, num_instances))

    for i in range(num_instances):
        for j in range(i):
            point_i = dataset.iloc[i].values
            point_j = dataset.iloc[j].values
            distance_matrix[i, j] = distance_points(point_i, point_j, p)

    return distance_matrix

# testing the function
distance_matrix = calculate_distance_matrix(dataFrame, p_value)
print("Lower Triangular Distance Matrix:")
print(distance_matrix)

Lower Triangular Distance Matrix:
[[   0.            0.            0.         ...    0.
     0.            0.        ]
 [  50.27772867    0.            0.         ...    0.
     0.            0.        ]
 [ 500.19889044  550.0745404     0.         ...    0.
     0.            0.        ]
 ...
 [2000.45537066 1950.36476076 2500.16825434 ...    0.
     0.            0.        ]
 [1450.35113679 1400.25685144 1950.08390845 ...  550.11712389
     0.            0.        ]
 [1650.34887221 1600.2636814  2150.09878145 ...  350.11628068
   200.06443962    0.        ]]


## d. The function closest_points 
- Write a function closest_points that, given a distance matrix, find the pair of the closest points. The function will take as parameter the distance
matrix and will return a tuple of the coordinates.


In [149]:
def closest_points(distance_matrix):
    num_instances = len(distance_matrix)
    min_distance = float('inf')
    closest_pair = None

    for i in range(num_instances):
        for j in range(i):
            if distance_matrix[i, j] < min_distance:
                min_distance = distance_matrix[i, j]
                closest_pair = (i, j)

    return closest_pair

# testing the function
closest_pair = closest_points(distance_matrix)
print(f"Closest pair: {closest_pair}")
print(f"Distance: {distance_matrix[closest_pair]}")



Closest pair: (278, 215)
Distance: 0.5385164807134515


# 3. Agglomerative Clustering
## a. The function update_distance_matrix_complete 
- Write a function update_distance_matrix_single that updates the distance matrix using the single linkage. The function will take as parameter the
distance matrix and the coordinates of the points / clusters that are merged, and will return the updated matrix.
Note: You should remove the rows and columns that are no longer used after the merge.


In [150]:
def update_distance_matrix_single(distance_matrix, merge_coordinates):
    i, j = merge_coordinates
    num_clusters = len(distance_matrix)

    # Calculate distances to the new cluster formed by merging i and j
    new_distances = np.minimum(distance_matrix[i, :], distance_matrix[j, :])

    # Update the distances in the new cluster's row and column
    distance_matrix[i, :] = new_distances
    distance_matrix[:, i] = new_distances

    # Remove the row and column corresponding to the merged cluster j
    distance_matrix = np.delete(distance_matrix, j, axis=0)
    distance_matrix = np.delete(distance_matrix, j, axis=1)

    return distance_matrix


# testing the function
updated_distance_matrix = update_distance_matrix_single(distance_matrix, closest_pair)
print("Updated Distance Matrix (Single Linkage):")
print(updated_distance_matrix)


Updated Distance Matrix (Single Linkage):
[[   0.            0.            0.         ...    0.
     0.            0.        ]
 [  50.27772867    0.            0.         ...    0.
     0.            0.        ]
 [ 500.19889044  550.0745404     0.         ...    0.
     0.            0.        ]
 ...
 [2000.45537066 1950.36476076 2500.16825434 ...    0.
     0.            0.        ]
 [1450.35113679 1400.25685144 1950.08390845 ...  550.11712389
     0.            0.        ]
 [1650.34887221 1600.2636814  2150.09878145 ...  350.11628068
   200.06443962    0.        ]]


## b. The function update_distance_matrix_complete
- Write a function update_distance_matrix_complete that updates the distance matrix using the complete linkage. The function will take as parameter
the distance matrix and the coordinates of the points / clusters that are merged, and will return the updated matrix.


In [151]:
def update_distance_matrix_complete(distance_matrix, merge_coordinates):
    i, j = merge_coordinates
    num_clusters = len(distance_matrix)

    # Calculate distances to the new cluster formed by merging i and j
    new_distances = np.maximum(distance_matrix[i, :], distance_matrix[j, :])

    # Update the distances in the new cluster's row and column
    distance_matrix[i, :] = new_distances
    distance_matrix[:, i] = new_distances

    # Remove the row and column corresponding to the merged cluster j
    distance_matrix = np.delete(distance_matrix, j, axis=0)
    distance_matrix = np.delete(distance_matrix, j, axis=1)

    return distance_matrix

# testing the function
updated_distance_matrix = update_distance_matrix_complete(distance_matrix, closest_pair)
print("Updated Distance Matrix (Complete Linkage):")
print(updated_distance_matrix)


Updated Distance Matrix (Complete Linkage):
[[   0.            0.            0.         ...    0.
     0.            0.        ]
 [  50.27772867    0.            0.         ...    0.
     0.            0.        ]
 [ 500.19889044  550.0745404     0.         ...    0.
     0.            0.        ]
 ...
 [2000.45537066 1950.36476076 2500.16825434 ...    0.
     0.            0.        ]
 [1450.35113679 1400.25685144 1950.08390845 ...  550.11712389
     0.            0.        ]
 [1650.34887221 1600.2636814  2150.09878145 ...  350.11628068
   200.06443962    0.        ]]


## c. The function update_distance_matrix_average 
- Write a function update_distance_matrix_average that updates the distance matrix using the average linkage. The function will take as parameter,
the distance matrix and the coordinates of the points / clusters that are merged, and will return the updated matrix.
Note: For full grading of the exercise, you should use the cluster sizes in the update.


In [152]:
def update_distance_matrix_average(distance_matrix, merge_coordinates):
    i, j = merge_coordinates
    num_clusters = len(distance_matrix)

    # Calculate the size of the new cluster formed by merging i and j
    cluster_size = 2

    # Calculate the average distances to the new cluster
    new_distances = (distance_matrix[i, :] + distance_matrix[j, :]) / cluster_size

    # Update the distances in the new cluster's row and column
    distance_matrix[i, :] = new_distances
    distance_matrix[:, i] = new_distances

    # Remove the row and column corresponding to the merged cluster j
    distance_matrix = np.delete(distance_matrix, j, axis=0)
    distance_matrix = np.delete(distance_matrix, j, axis=1)

    return distance_matrix

# testing the function
updated_distance_matrix = update_distance_matrix_average(distance_matrix, closest_pair)
print("Updated Distance Matrix (Average Linkage):")
print(updated_distance_matrix)


Updated Distance Matrix (Average Linkage):
[[   0.            0.            0.         ...    0.
     0.            0.        ]
 [  50.27772867    0.            0.         ...    0.
     0.            0.        ]
 [ 500.19889044  550.0745404     0.         ...    0.
     0.            0.        ]
 ...
 [2000.45537066 1950.36476076 2500.16825434 ...    0.
     0.            0.        ]
 [1450.35113679 1400.25685144 1950.08390845 ...  550.11712389
     0.            0.        ]
 [1650.34887221 1600.2636814  2150.09878145 ...  350.11628068
   200.06443962    0.        ]]


## d. The function calculate_dendogram_height
- Write a function calculate_dendogram_height that calculates the height of the dendogram associated with the newly formed cluster. You should use
the formula based on the average of the distances of the points inside the cluster.
Note: For full grading of the exercise, you should use the heights of the dendogram associated with the two clusters that are going to be merged.


In [153]:
def calculate_dendrogram_height(height1, size1, height2, size2):
    height = (height1 * size1 + height2 * size2) / (size1 + size2)
    return height


## e. The function calculate_dendogram_height_average 
- Write a function calculate_dendogram_heigh_average that calculates the height in the case of average linkage. You should use only the previously
calculated dendogram heights and the updated distance matrix.


In [154]:
def calculate_dendrogram_height_average(height1, size1, height2, size2):
    height_average = (height1 * size1 + height2 * size2) / (size1 + size2)
    return height_average


## f. The function agglomerative_clustering
- Write a function `agglomerative_custering` , which will take as parameters the dataframe `df` , the number of desired clusters `nclusters` , the linkage
and the Minkowski distance type p (use Euclidean by default). The number of desired clusters acts as a stopping criteria: the merging should stop if
the current number of clusters = `nclusters` .
- The function should return the following dictionary:
```py
{
"membership" : membership vector, i.e. a vector which for each instance contains the cluster it belongs to
"dendogram_heights" : {
"cluster1" : height1,
"cluster2" : height2,
...
}
}
```
`dendogram_heights` is a dictionary where each key represents the cluster formed after the merge in one iteration. The cluster is represented as a set /
list of indices associated with the instances belonging to his cluster. The values of the dictionary are the calculated heights.

In [155]:
def agglomerative_clustering(df, nclusters, linkage, p=2):
    # Initialize the distance matrix
    distance_matrix = calculate_distance_matrix(df, p)

    # Initialize the cluster membership vector
    membership = np.arange(len(df))

    # Initialize the dendrogram heights dictionary
    dendrogram_heights = {}

    # Perform agglomerative clustering until the desired number of clusters is reached
    while len(np.unique(membership)) > nclusters:
        # Find the pair of closest points
        if(closest_points(distance_matrix) == None):
            break
        
        closest_pair = closest_points(distance_matrix)

        # Merge the closest pair of points
        merged_cluster = np.union1d(membership == closest_pair[0], membership == closest_pair[1])

        # Update the distance matrix based on the chosen linkage
        if linkage == 'single':
            distance_matrix = update_distance_matrix_single(distance_matrix, closest_pair)
        elif linkage == 'complete':
            distance_matrix = update_distance_matrix_complete(distance_matrix, closest_pair)
        elif linkage == 'average':
            distance_matrix = update_distance_matrix_average(distance_matrix, closest_pair)

        # Update the cluster membership vector
        membership = np.where(merged_cluster[:, np.newaxis], len(df) + 1, membership)

        # Calculate the dendrogram height for the merged cluster
        height1 = dendrogram_heights.get(closest_pair[0], 0)
        height2 = dendrogram_heights.get(closest_pair[1], 0)
        size1 = np.sum(membership == closest_pair[0])
        size2 = np.sum(membership == closest_pair[1])
        dendrogram_heights[len(df) + 1] = calculate_dendrogram_height_average(height1, size1, height2, size2)

    return {'membership': membership, 'dendrogram_heights': dendrogram_heights}

# testing the function
result = agglomerative_clustering(dataFrame, 3, 'single')
print("Membership Vector:")
print(result['membership'])
print("Dendrogram Heights:")
print(result['dendrogram_heights'])




Membership Vector:
[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
   36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
   54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
   72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
   90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
  108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
  126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
  144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
  162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
  180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
  198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
  216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
  234 235 236 237 2